<a href="https://colab.research.google.com/github/tylerfeldman321/GANs-for-Medical-Data/blob/main/ECG_Synthesis_and_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
%matplotlib inline

In [ ]:
import os
if not os.path.exists('/content/drive/MyDrive/MUSER'):
    os.mkdir('/content/drive/MyDrive/MUSER')

%cd '/content/drive/MyDrive/MUSER'
if not os.path.exists('ECG-Synthesis-and-Classification'):
    !git clone https://github.com/tylerfeldman321/ECG-Synthesis-and-Classification.git

In [ ]:
%cd '/content/drive/MyDrive/MUSER'

%cp -R 'ECG-Synthesis-and-Classification/ecg_gan/results' 'results_gan'
%cp -R 'ECG-Synthesis-and-Classification/ecg_classification/results' 'results_classification'

%cp 'ECG-Synthesis-and-Classification/ecg_classification/wesad_train_full_heartbeat_subject_id_labels.csv' 'wesad_train_full_heartbeat_subject_id_labels.csv'
%cp 'ECG-Synthesis-and-Classification/ecg_classification/wesad_test_full_heartbeat_subject_id_labels.csv' 'wesad_test_full_heartbeat_subject_id_labels.csv'

!rm -rf 'ECG-Synthesis-and-Classification'

!git clone https://github.com/tylerfeldman321/ECG-Synthesis-and-Classification.git

%cp -R 'results_gan' 'ECG-Synthesis-and-Classification/ecg_gan/results'
%cp -R 'results_classification' 'ECG-Synthesis-and-Classification/ecg_classification/results'

%cp 'wesad_train_full_heartbeat_subject_id_labels.csv' 'ECG-Synthesis-and-Classification/ecg_classification/wesad_train_full_heartbeat_subject_id_labels.csv'
%cp 'wesad_test_full_heartbeat_subject_id_labels.csv' 'ECG-Synthesis-and-Classification/ecg_classification/wesad_test_full_heartbeat_subject_id_labels.csv'

!rm -rf 'results_gan'
!rm -rf 'results_classification'

In [ ]:
%cd '/content/drive/MyDrive/MUSER/ECG-Synthesis-and-Classification/ecg_gan'
import pandas as pd

# train_path = 'mitbih_train.csv'
# test_path = 'mitbih_test.csv'
train_path = 
test_path = 

df_dataset = pd.read_csv(train_path)

In [ ]:
df_train = pd.read_csv(train_path, header=None)
df_test = pd.read_csv(test_path, header=None)
df_train.rename(columns={len(df_dataset.columns)-1: 'class'}, inplace=True)
df_test.rename(columns={len(df_dataset.columns)-1: 'class'}, inplace=True)

id_to_label = {
    0: "Normal",
    1: "Artial Premature",
    2: "Premature ventricular contraction",
    3: "Fusion of ventricular and normal",
    4: "Fusion of paced and normal"
}

df_train['label'] = df_train.iloc[:, -1].map(id_to_label)
df_test['label'] = df_test.iloc[:, -1].map(id_to_label)

In [ ]:
df_train.head()

In [ ]:
df_train.to_csv('data.csv', index=False)

In [ ]:
df_train['label'].value_counts()

In [ ]:
# Plot examples of the input data
import matplotlib.pyplot as plt
indexes = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120]
for index in indexes:
    row = df_dataset.iloc[index, :-2]
    row.plot(label=f'ECG Sample {index}')
    plt.xlabel('Time Index')
    plt.ylabel('Magnitude')
    plt.title('ECG Samples')

In [ ]:
!python train.py

In [ ]:
%cd '/content/drive/MyDrive/MUSER/ECG-Synthesis-and-Classification/ecg_gan'
results_subdir = 'Subject2-Month01-Day21-Hour16-Minute31'
test_csv_filename = 'wesad_test_full_heartbeat_subject_2.csv'

In [ ]:
# Plot generator and discriminator losses
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt
results_dir = os.path.join('results', results_subdir)

with open(os.path.join(results_dir, 'Errors.pickle'), 'rb') as f:
    losses = pickle.load(f)
    print(losses['g'])
    print(losses['d'])
    epoch = np.arange(0, len(losses['g']), 1)
    plt.plot(epoch, losses['g'], label='Generator Loss')
    plt.plot(epoch, losses['d'], label='Discriminator Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Generator and Discriminator Loss vs. Epoch')

In [12]:
from sklearn import metrics

def mmd_rbf(X, Y, gamma=1.0):
    """MMD using rbf (gaussian) kernel (i.e., k(x,y) = exp(-gamma * ||x-y||^2 / 2))
    Arguments:
        X {[n_sample1, dim]} -- [X matrix]
        Y {[n_sample2, dim]} -- [Y matrix]
    Keyword Arguments:
        gamma {float} -- [kernel parameter] (default: {1.0})
    Returns:
        [scalar] -- [MMD value]
    """
    XX = metrics.pairwise.rbf_kernel(X, X, gamma)
    YY = metrics.pairwise.rbf_kernel(Y, Y, gamma)
    XY = metrics.pairwise.rbf_kernel(X, Y, gamma)
    return XX.mean() + YY.mean() - 2 * XY.mean()

In [ ]:
# Plot generated samples for each epoch
import glob
import numpy as np
import pandas as pd

results_dir = os.path.join('results', results_subdir)
test_csv_path = os.path.join(test_csv_filename)
df = pd.read_csv(test_csv_path)
test_data = df.to_numpy()[:, :-1]

data_files = glob.glob(os.path.join(results_dir, '*.npy'))

plot_mmd = True
mmd_scores = []
epochs = []
for data_file in data_files:
    epoch = str(os.path.basename(data_file).split('-')[1])
    with open(data_file, 'rb') as f:
        data = np.load(f)
        plt.plot(data)
        plt.xlabel('Time Index')
        plt.ylabel('Magnitude')
        title = 'Epoch' + ' ' + epoch
        plt.title(title)
        plt.show()

        if plot_mmd:
            mmd = mmd_rbf(test_data, data.T)
            print(f"Epoch: {epoch}, mmd: {mmd}")
            epochs.append(int(epoch))
            mmd_scores.append(mmd)

if plot_mmd:
    print(epochs)
    print(mmd_scores)
    plt.figure()
    plt.plot(epochs, mmd_scores, 'b-')
    plt.title('MMD Scores Over Time')
    plt.xlabel('Epoch')
    plt.ylabel('MMD Score')
    plt.show()

    plt.figure()
    plt.plot(epochs, np.log(mmd_scores), 'b-')
    plt.title('MMD Scores Over Time')
    plt.xlabel('Epoch')
    plt.ylabel('MMD Score')
    plt.show()

## Classification

In [ ]:
import pandas as pd
%cd '/content/drive/MyDrive/MUSER/ECG-Synthesis-and-Classification/ecg_classification'
df_train = pd.read_csv('wesad_train_full_heartbeat_subject_id_labels.csv', header=None)
df_test = pd.read_csv('wesad_test_full_heartbeat_subject_id_labels.csv', header=None)
print(df_train.isnull().values.any())
print(df_test.isnull().values.any())
# df_mitbih = pd.concat([df_train, df_test], axis=0)
df_train.rename(columns={187: 'class'}, inplace=True)
df_test.rename(columns={187: 'class'}, inplace=True)

id_to_label = {
    0: "S0",
    1: "S1",
    2: "S2",
    3: "S3",
    4: "S4",
    5: "S5",
    6: "S6",
    7: "S7",
    8: "S8",
    9: "S9",
    10: "S10",
    11: "S11",
    12: "S12",
    13: "S13",
    14: "S14",
    15: "S15",
    16: "S16",
    17: "S17",
}


df_train['label'] = df_train.iloc[:, -1].map(id_to_label)
df_test['label'] = df_test.iloc[:, -1].map(id_to_label)

print(df_train.head())
print(df_train.info())
print(df_test.info())

print(df_train.isnull().values.any())
print(df_test.isnull().values.any())

df_train.to_csv('data_train.csv', index=False)
df_test.to_csv('data_test.csv', index=False)

In [ ]:
df_train = pd.read_csv('data_train.csv')
df_test = pd.read_csv('data_test.csv')

In [ ]:
print(df_train['label'].value_counts())
print(df_test['label'].value_counts())

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

df_all = pd.concat([df_train, df_test], axis=0)
percentages = [count / df_all.shape[0] * 100 for count in df_all['label'].value_counts()]

fig, ax = plt.subplots(figsize=(26, 8))
sns.countplot(
    x=df_all['label'],
    ax=ax,
    palette="bright",
    order=df_all['label'].value_counts().index
)
ax.set_xticklabels(ax.get_xticklabels(), rotation=15);

for percentage, count, p in zip(
    percentages,
    df_all['label'].value_counts(sort=True).values,
    ax.patches):
    
    percentage = f'{np.round(percentage, 2)}%'
    x = p.get_x() + p.get_width() / 2 - 0.4
    y = p.get_y() + p.get_height()
    ax.annotate(str(percentage)+" / "+str(count), (x, y), fontsize=12, fontweight='bold')
    
plt.savefig('data_dist.png', facecolor='w', edgecolor='w', format='png',
        transparent=False, bbox_inches='tight', pad_inches=0.1)
plt.savefig('data_dist.svg', facecolor='w', edgecolor='w', format='svg',
        transparent=False, bbox_inches='tight', pad_inches=0.1)

In [ ]:
%%time
signals = [' '.join(df_mitbih.iloc[i, :-1].apply(str).values) for i in range(df_mitbih.shape[0])]
y = df_mitbih.iloc[:, -1].values.tolist()
print(len(signals), len(y))

print(f'data has {len(set([sig for line in signals for sig in line.split()]))} out of 16 372 411 unique values.')

In [ ]:
!python train.py